# Save pain ratings for each subject in a loop

In [8]:
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
from IPython import display
import scipy.io as scio

data_dir = "../../../Data/EEG DATA/"
proc_dir = "../../../Data/Processed Data/"
save_dir = "./pain_ratings_and_stim_labels/"

def delete_mult_list_els(list_object, indices):
    ''' Function for deleting multiple elements from a list'''
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)## define functions for extracting relevant epochs

In [9]:
#### all subject file names in directory for reference
sub_ids_lst = [
 'C26',
 'C25',
 # '006', Jumbled markers
 # '007', #32ch
 # '008', #32ch
 # '009', #32ch
 # '010', #32ch
 # '013', #32ch
 # '014', #32ch
 # '015', #32ch
 # '016', X
 # '017', X
 '018',
 # '019', X
 '020',
 '021',
 '022',
 '023',
 '024',
 # '025', X
 # '026', X
 '027',
 # '028', X
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 # '044', #TIMESTAMPS MISMATCH
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 'C1.',
 'C10',
 # 'C11', #empty
 'C12', 
 'C13',
 'C14',
 'C15',
 'C16',
 # 'C17', #TIMESTAMPS MISMATCH
 'C18',
 'C19',
 # 'C2.', #TIMESTAMPS MISMATCH
 'C20',
 'C21',
 'C22',
 # 'C23', X
 'C24',
 # 'C3.', #TIMESTAMPS MISMATCH
 # 'C4.', X
 # 'C5.', #TIMESTAMPS MISMATCH
 'C6.',
 'C7.',
 'C8.',
 'C9.']

In [10]:
sub_ids_lst=['C12']

In [11]:
for folder in os.listdir(data_dir):
    sub_id=''; acq_id=''
    ########################################## Identify subject files ##########################################
    save_fname = folder[:3]+'_pain_ratings.mat'
    if ( (not folder.startswith('.')) and (folder[:3] in sub_ids_lst)):
        edf_dir = folder
        sub_id = folder[:3]
        print(sub_id)
    else:
        continue
    # print(f"Opened folder '/{folder}/'!")
    
    # Load in drop log for bad trials
    drop_log = scio.loadmat(os.path.join(proc_dir,sub_id+'_drop_log.mat'))
    drop_log = drop_log['drop_log'].tolist()
    if len(drop_log) > 0: drop_log = drop_log[0]
    
    print(f"*drop_log.size = {len(drop_log)}*")

    ############################ Open pain rating excel file and extract stimulus labels and pain ratings ############################
    xfname=""
    for file in os.listdir(data_dir+edf_dir):
        if file.endswith(".xlsx"):
            xfname = file
    df = pd.read_excel(os.path.join(data_dir,edf_dir,xfname), sheet_name=0) 

    lower_back_flag = 0
    try:
        if isinstance(df['Unnamed: 2'].tolist().index('LOWER BACK '), int):
            column_back_id = df['Unnamed: 1'].tolist().index('LOWER BACK ')
            excel_list_hand = df['Unnamed: 1'][3:column_back_id].tolist()
        if ' 32 guage (3) ' in excel_list_hand:
            # index where rows switch to back
            excel_list_hand_new = []
            for idx, el in enumerate(excel_list_hand):
                if el == ' 32 guage (3) ':
                    excel_list_hand_new.append(3)
                elif el == 'PM (4)':
                    excel_list_hand_new.append(4)
            # back rows
            excel_list_back = df['Unnamed: 1'][column_back_id:].tolist()
            excel_list_back_new = []
            for idx, el in enumerate(excel_list_back):
                if el == ' 32 guage (3) ':
                    excel_list_back_new.append(5)
                elif el == 'PM (4)':
                    excel_list_back_new.append(6)

        lower_back_flag = 1
    except: pass

    if lower_back_flag:
        # concatenate lists
        excel_list = excel_list_hand_new + excel_list_back_new
    else:
        # concatenate lists
        excel_list = df['Unnamed: 1'][3:].tolist() 
    pain_ratings = excel_list
    
    # check if excel_list contains nans which happens for some reason
    if np.any(np.isnan(pain_ratings)):
        excel_nans = np.where(np.isnan(pain_ratings))
        excel_nans=excel_nans[0].tolist()
        print("Excel NaNs:",excel_nans)
        delete_mult_list_els(pain_ratings,excel_nans)
    
    # drop bad epochs
    delete_mult_list_els(pain_ratings,drop_log)
    print(f"*pain_ratings size after drop = {len(pain_ratings)}*")

    # save pain ratings
    print("Saving pain ratings as mat file...\n")
    mdic = {"pain_ratings": pain_ratings }
    scio.savemat(save_dir+sub_id[:3]+'_pain_ratings.mat', mdic)

    # Load in stim labels
    stim_labels = scio.loadmat(os.path.join(proc_dir,sub_id+'_stim_labels.mat'))
    if sub_id=='C2.':
        stim_labels = stim_labels['stim_labels'].tolist()
        stim_labels_new = [el[0] for el in stim_labels]
        stim_labels=stim_labels_new
    else:
        stim_labels = stim_labels['stim_labels'].tolist()[0]
    print(f"*stim_labels size = {len(stim_labels)}*")
   
    # Load in epo_times
    epo_times = scio.loadmat(os.path.join(proc_dir,sub_id+'_epo_times.mat'))
    epo_times = epo_times['epo_times'].tolist()
    print(f"*epo_times size = {len(epo_times)}*")
   
    if len(stim_labels) != len(pain_ratings):
        # drop bad epochs
        delete_mult_list_els(stim_labels,drop_log)
        print(f"*stim_labels size after drop = {len(stim_labels)}*")

        # save
        print("Saving stim_labels as mat file...")
        mdic = {"stim_labels": stim_labels }
        scio.savemat(save_dir+sub_id[:3]+'_stim_labels.mat', mdic)

    if len(epo_times) != len(pain_ratings):
        # drop bad epochs
        epo_times = np.delete(epo_times,drop_log,axis=0)
        print(f"*epo_times size after drop = {len(epo_times)}*")
        
        # save
        print("Saving epo_times as mat file...")
        mdic = {"epo_times": epo_times}
        scio.savemat(save_dir+sub_id[:3]+'_epo_times.mat', mdic)
    
    print(f"\nDone.\n\n")
    # break

C12
*drop_log.size = 0*
*pain_ratings size after drop = 60*
Saving pain ratings as mat file...

*stim_labels size = 60*
*epo_times size = 60*

Done.


